In [27]:
# Data manipulation
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Models
import pmdarima as pm
from pmdarima.arima import auto_arima
from arch import arch_model

# Evaluation
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

In [2]:
# Read and preprocess data

aapl = pd.read_csv("data_files/AAPL_combined.csv.gz")
nee = pd.read_csv("data_files/NEE_combined.csv.gz")
lly = pd.read_csv("data_files/LLY_combined.csv.gz")


def data_preprocess(df):
    # Rename columns
    df.rename(columns={'Unnamed: 0': 'timestamp',
                       '1. open': 'open',
                       '2. high': 'high',
                       '3. low': 'low',
                       '4. close': 'close',
                       '5. volume': 'volume'}, inplace=True)
    
    # Drop unnecessary columns
    if 'Unnamed: 0.1' in df.columns.tolist():
        df.drop(columns=['Unnamed: 0.1'], inplace=True)

    # Handle data types
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    return df

aapl = data_preprocess(aapl)
nee = data_preprocess(nee)
lly = data_preprocess(lly)

In [3]:
# Compute daily volatility

def daily_volatility(df):
    """
    Aggregates to daily level and calculates volatility based on the 
    logic: sqrt(log(1 + (close - open) / open))
    """
    # Select columns and drop NAs
    df = df[['timestamp', 'open', 'close']].dropna().copy()
    
    # Convert timestamp to datetime and extract the date
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    
    # Group by date to get first open and last close
    daily = df.groupby('date').agg(
        open=('open', 'first'),
        close=('close', 'last')
    ).reset_index()
    
    # Calculate log return
    # equivalent to R: log(1 + (close - open) / open) -> log(close / open)
    daily['log_return'] = np.log(1 + (daily['close'] - daily['open']) / daily['open'])
    
    # Calculate volatility
    # Note: If log_return is negative, sqrt will result in NaN. 
    # We use np.sqrt where valid, otherwise NaN.
    daily['volatility'] = daily['log_return'].apply(lambda x: np.sqrt(x) if x > 0 else np.nan)
    
    # Return only date and volatility, dropping any rows where calculation failed (NaNs)
    return daily[['date', 'volatility']].dropna()

aapl_vol = daily_volatility(aapl)
nee_vol = daily_volatility(nee)
lly_vol = daily_volatility(lly)


In [4]:
# Compute daily returns

def get_daily_returns(df):

    df = df[['timestamp', 'open', 'close']].dropna().copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    
    # Aggregation
    daily = df.groupby('date').agg(
        open=('open', 'first'),
        close=('close', 'last')
    ).reset_index()
    
    # Calculate Log Return
    daily['log_return'] = np.log(daily['close'] / daily['open'])
    daily['scaled_return'] = daily['log_return'] * 100
    
    return daily[['date', 'scaled_return']].dropna()

aapl_returns = get_daily_returns(aapl)
nee_returns = get_daily_returns(nee)
lly_returns = get_daily_returns(lly)

# Build  Baseline Models

### EWMA

In [17]:
class EWMAResults:

    def __init__(self, data, lambda_val):
        self._data = data
        self._lambda = lambda_val
    
        self.conditional_volatility = data.ewm(alpha=lambda_val, adjust=False).mean()
        self.resid = data - self.conditional_volatility

    def summary(self):
        print("EWMA Model Results")
        print("==================")
        print(f"Lambda (alpha): {self._lambda}")
        print(f"Last Volatility: {self.conditional_volatility.iloc[-1]:.5f}")
        return ""

    def forecast(self, horizon=1):
        last_vol = self.conditional_volatility.iloc[-1]

        class ForecastObj:
            def __init__(self, val, h):
                self.volatility = pd.Series([val] * h, name='h.1')
                self.variance = self.volatility ** 2
        
        return ForecastObj(last_vol, horizon)

In [24]:
def fit_ewma(df, lambda_):

    series = df['volatility'] 
    
    results = EWMAResults(series, lambda_)
    
    return results

In [25]:
aapl_ewma = fit_ewma(aapl_vol, lambda_ = 0.05)
nee_ewma = fit_ewma(nee_vol, lambda_= 0.01)
lly_ewma = fit_ewma(lly_vol, lambda_ = 0.01)

### ARIMA

In [7]:
def fit_arima(df):

    series = df['volatility'].values

    model = auto_arima(series, seasonal=False, error_action='ignore', suppress_warnings=True)

    return model

In [8]:
aapl_arima = fit_arima(aapl_vol)
nee_arima = fit_arima(nee_vol)
lly_arima = fit_arima(lly_vol)

### GARCH

In [33]:
def fit_garch_pq(df, p, q):    
    # Model
    model = arch_model(df['volatility'], vol='Garch', p=p, q=q, mean='Zero')
    results = model.fit(disp='off')

    return results

In [36]:
aapl_garch = fit_garch_pq(aapl_vol, 3, 2)
nee_garch = fit_garch_pq(nee_vol, 2, 2)
lly_garch = fit_garch_pq(lly_vol, 1, 1)

/Users/abigailalpert/anaconda3/envs/cse217a/lib/python3.10/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002121. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/Users/abigailalpert/anaconda3/envs/cse217a/lib/python3.10/site-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002037. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids

# Model Comparison

In [35]:
def compare_models(ticker_name, ewma_res, arima_res, garch_res, original_data):
    """
    Compares EWMA, ARIMA, and GARCH models.
    """
    metrics = {}
    
    # EWMA
    ewma_rmse = np.sqrt(np.mean(ewma_res.resid**2))
    metrics['EWMA'] = {'AIC': np.nan, 'BIC': np.nan, 'RMSE': ewma_rmse}

    # ARIMA

    arima_rmse = np.sqrt(np.mean(arima_res.resid()**2))
    metrics['ARIMA'] = {'AIC': arima_res.aic(), 'BIC': arima_res.bic(), 'RMSE': arima_rmse}

    # GARCH
    garch_rmse = np.sqrt(mean_squared_error(original_data, garch_res.conditional_volatility))
    metrics['GARCH'] = {'AIC': garch_res.aic, 'BIC': garch_res.bic, 'RMSE': garch_rmse}
    
    # Create DataFrame
    df_metrics = pd.DataFrame(metrics).T
    print(f"--- Model Comparison for {ticker_name} ---")
    display(df_metrics)
    return df_metrics

# Run the comparison
aapl_comparison = compare_models("AAPL", aapl_ewma, aapl_arima, aapl_garch, aapl_vol['volatility'])
nee_comparison = compare_models("NEE", nee_ewma, nee_arima, nee_garch, nee_vol['volatility'])
lly_comparison = compare_models("LLY", lly_ewma, lly_arima, lly_garch, lly_vol['volatility'])

--- Model Comparison for AAPL ---


,AIC,BIC,RMSE
EWMA,NaN,NaN,0.042498
ARIMA,-2606.102279,-2596.799136,0.045038
GARCH,-1260.083671,-1246.125082,0.045496


--- Model Comparison for NEE ---


,AIC,BIC,RMSE
EWMA,NaN,NaN,0.044844
ARIMA,-2525.690828,-2507.231926,0.044283
GARCH,-1265.807831,-1251.963654,0.045147


--- Model Comparison for LLY ---


,AIC,BIC,RMSE
EWMA,NaN,NaN,0.048349
ARIMA,-2485.155743,-2466.643577,0.046520
GARCH,-1205.989533,-1173.593243,0.047553


Based on AIC and BIC, ARIMA is better for AAPL, NEE, and LLY.

Based on RMSE, EWMA is the best for AAPL, ARIMA is best for NEE, and ARIMA is best for LLY.

Due to these results, we can conclude that ARIMA is the best baseline model